# Multi-node multi-GPU example on AWS using dask-cloudprovider

[Dask Cloud Provider](https://cloudprovider.dask.org/en/latest/) is a native cloud integration for dask. It helps manage Dask clusters on different cloud platforms. In this notebook, we will look at how we can use the package to set-up a AWS cluster and run a multi-node multi-GPU (MNMG) example with [RAPIDS](https://rapids.ai/). RAPIDS provides a suite of libraries to accelerate data science pipelines on the GPU entirely. This can be scaled to multiple nodes using Dask as we will see through this notebook. 

### Local Environment Setup

1. Before running the notebook, ensure you have installed RAPIDS locally. Follow the instructions in [README.md](https://github.com/rapidsai/cloud-ml-examples/blob/main/aws/README.md) to install RAPIDS locally.

2. Install and set up AWS CLI using the following commands:

```bash
pip install awscli
aws configure

```
The list of packages needed for this notebook is listed in the cell below - uncomment and run the cell to set it up.

In [ ]:
# !pip install "dask-cloudprovider[aws]"

In [ ]:
import math
from datetime import datetime

import cudf
import dask
import dask_cudf
import numpy as np

from cuml.dask.common import utils as dask_utils
from cuml.dask.ensemble import RandomForestRegressor
from cuml.metrics import mean_squared_error
from dask_cloudprovider.aws import EC2Cluster
from dask.distributed import Client
from dask_ml.model_selection import train_test_split
from dateutil import parser
import configparser, os, contextlib

## AWS Cluster Set-up
We'll now setup the [EC2Cluster](https://cloudprovider.dask.org/en/latest/aws.html#elastic-compute-cloud-ec2) from Dask Cloud Provider. To do this, you'll first need to run `aws configure` and ensure the credentials are updated. [Learn more about the setup](https://cloudprovider.dask.org/en/latest/aws.html#authentication). The API also expects a security group that allows access to ports 8786-8787 and all traffic between instances in the security group. If you do not pass a group here, dask cloud provider will create one for you.

Note: Make sure you have permissions in the account to create the necessary resources.

In [ ]:
def get_aws_credentials():
    parser = configparser.RawConfigParser()
    parser.read(os.path.expanduser('~/.aws/config'))
    config = parser.items('default')
    parser.read(os.path.expanduser('~/.aws/credentials'))
    credentials = parser.items('default')
    all_credentials = {key.upper(): value for key, value in [*config, *credentials]}
    with contextlib.suppress(KeyError):
        all_credentials["AWS_REGION"] = all_credentials.pop("REGION")
    return all_credentials

In [ ]:
n_workers = 2
n_gpus_per_worker = 4
security_group = "sg-dask"

In [ ]:
cluster = EC2Cluster(env_vars=get_aws_credentials(),
                     instance_type="p3.8xlarge", # 4 V100 GPUs
                     docker_image="rapidsai/rapidsai:21.08-cuda11.0-runtime-ubuntu18.04-py3.8",
                     worker_class="dask_cuda.CUDAWorker",
                     worker_options = {'rmm-managed-memory':True},
                     security_groups=[security_group],
                     docker_args = '--shm-size=256m',
                     n_workers=n_workers,
                     security=False,
                     availability_zone="")

## Client set up

The cells below create a [Dask Client](https://distributed.dask.org/en/latest/client.html) with the cluster we defined earlier in the notebook accessing the VM. Once we have the client, we need to wait for the workers to be up and running, we do so by adding 

```python
client.wait_for_workers(n_workers)
```

In [ ]:
client = Client(cluster)
client

In [ ]:
%%time
client.wait_for_workers(n_workers*n_gpus_per_worker)
client

## Machine Learning Workflow

Once workers become available, we can now run the rest of our workflow:

- read and clean the data
- add features
- split into training and validation sets
- fit a Random Forest model
- predict on the validation set
- compute RMSE

#### 1. Read and Clean Data

The data needs to be cleaned up before it can be used in a meaningful way. We verify the columns have appropriate datatypes to make it ready for computation using cuML.

In [2]:
# create a list of all columns & dtypes the df must have for reading
col_dtype = {
    'VendorID': 'int32',
    'tpep_pickup_datetime': 'datetime64[ms]',
    'tpep_dropoff_datetime': 'datetime64[ms]',
    'passenger_count': 'int32',
    'trip_distance': 'float32',
    'pickup_longitude': 'float32',
    'pickup_latitude': 'float32',
    'RatecodeID': 'int32',
    'store_and_fwd_flag': 'int32',
    'dropoff_longitude': 'float32',
    'dropoff_latitude': 'float32',
    'payment_type':'int32',
    'fare_amount': 'float32',
    'extra':'float32',
    'mta_tax':'float32',
    'tip_amount': 'float32',
    'total_amount': 'float32',
    'tolls_amount': 'float32',
    'improvement_surcharge': 'float32',
}

In [ ]:
taxi_df = dask_cudf.read_csv("s3://nyc-tlc/trip data/yellow_tripdata_2016-02.csv",
                             storage_options={"anon": True},
                             dtype=col_dtype)

In [ ]:
#Dictionary of required columns and their datatypes
must_haves = {
    'pickup_datetime': 'datetime64[ms]',
    'dropoff_datetime': 'datetime64[ms]',
    'passenger_count': 'int32',
    'trip_distance': 'float32',
    'pickup_longitude': 'float32',
    'pickup_latitude': 'float32',
    'rate_code': 'int32',
    'dropoff_longitude': 'float32',
    'dropoff_latitude': 'float32',
    'fare_amount': 'float32'
}

In [ ]:
def clean(ddf, must_haves):
    # replace the extraneous spaces in column names and lower the font type
    tmp = {col:col.strip().lower() for col in list(ddf.columns)}
    ddf = ddf.rename(columns=tmp)

    ddf = ddf.rename(columns={
        'tpep_pickup_datetime': 'pickup_datetime',
        'tpep_dropoff_datetime': 'dropoff_datetime',
        'ratecodeid': 'rate_code'
    })
    
    ddf['pickup_datetime'] = ddf['pickup_datetime'].astype('datetime64[ms]')
    ddf['dropoff_datetime'] = ddf['dropoff_datetime'].astype('datetime64[ms]')

    for col in ddf.columns:
        if col not in must_haves:
            ddf = ddf.drop(columns=col)
            continue
        if ddf[col].dtype == 'object':
            # Fixing error: could not convert arg to str
            ddf = ddf.drop(columns=col)
        else:
            # downcast from 64bit to 32bit types
            # Tesla T4 are faster on 32bit ops
            if 'int' in str(ddf[col].dtype):
                ddf[col] = ddf[col].astype('int32')
            if 'float' in str(ddf[col].dtype):
                ddf[col] = ddf[col].astype('float32')
            ddf[col] = ddf[col].fillna(-1)
    
    return ddf

In [ ]:
taxi_df = taxi_df.map_partitions(clean, must_haves, meta=must_haves)

#### 2. Add Features

We'll add new features to the dataframe:

1. We can split the datetime column to retrive year, month, day, hour, day_of_week columns. Find the difference between pickup time and drop off time. 
2. Haversine Distance between the pick-up and drop-off coordinates.

In [ ]:
## add features

taxi_df['hour'] = taxi_df['pickup_datetime'].dt.hour.astype('int32')
taxi_df['year'] = taxi_df['pickup_datetime'].dt.year.astype('int32')
taxi_df['month'] = taxi_df['pickup_datetime'].dt.month.astype('int32')
taxi_df['day'] = taxi_df['pickup_datetime'].dt.day.astype('int32')
taxi_df['day_of_week'] = taxi_df['pickup_datetime'].dt.weekday.astype('int32')
taxi_df['is_weekend'] = (taxi_df['day_of_week']>=5).astype('int32')

#calculate the time difference between dropoff and pickup.
taxi_df['diff'] = taxi_df['dropoff_datetime'].astype('int32') - taxi_df['pickup_datetime'].astype('int32')
taxi_df['diff']=(taxi_df['diff']/1000).astype('int32')

taxi_df['pickup_latitude_r'] = taxi_df['pickup_latitude']//.01*.01
taxi_df['pickup_longitude_r'] = taxi_df['pickup_longitude']//.01*.01
taxi_df['dropoff_latitude_r'] = taxi_df['dropoff_latitude']//.01*.01
taxi_df['dropoff_longitude_r'] = taxi_df['dropoff_longitude']//.01*.01

taxi_df = taxi_df.drop('pickup_datetime', axis=1)
taxi_df = taxi_df.drop('dropoff_datetime', axis=1)

def haversine_dist(df):
    import cuspatial
    h_distance = cuspatial.haversine_distance(df['pickup_longitude'], df['pickup_latitude'], df['dropoff_longitude'], df['dropoff_latitude'])
    df['h_distance']= h_distance
    df['h_distance']= df['h_distance'].astype('float32')
    return df
taxi_df = taxi_df.map_partitions(haversine_dist)

#### 3. Split Data

In [ ]:
# Split into training and validation sets
X, y = taxi_df.drop(["fare_amount"], axis=1).astype('float32'), taxi_df["fare_amount"].astype('float32')
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True)

In [ ]:
workers = client.has_what().keys()
X_train, X_test, y_train, y_test = dask_utils.persist_across_workers(client,
                                                     [X_train, X_test, y_train, y_test],
                                                     workers=workers)

#### 4. Create and fit a Random Forest Model

In [ ]:
# create cuml.dask RF regressor
cu_dask_rf = RandomForestRegressor(ignore_empty_partitions=True)

In [ ]:
# fit RF model
cu_dask_rf = cu_dask_rf.fit(X_train, y_train)

#### 5. Predict on validation set

In [ ]:
#predict on validation set
y_pred = cu_dask_rf.predict(X_test)

#### 6. Compute RMSE

In [ ]:
# compute RMSE
score = mean_squared_error(y_pred.compute().to_array(), y_test.compute().to_array())
print("Workflow Complete - RMSE: ", np.sqrt(score))

### Resource Cleanup

In [ ]:
# Clean up resources
client.close()
cluster.close()

#### Learn More

- [Multi-node multi-GPU Random Forest example on Azure using dask-cloudprovider](https://github.com/rapidsai/cloud-ml-examples/blob/main/azure/notebooks/Azure-MNMG-RF.ipynb)

- [
Multi-Node Multi-GPU XGBoost example on Azure using dask-cloudprovider](https://github.com/rapidsai/cloud-ml-examples/blob/main/azure/notebooks/Azure-MNMG-XGBoost.ipynb)

- [Dask Cloud Provider](https://cloudprovider.dask.org/en/latest/)